In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
import statistics
from keras.models import Sequential
from keras.layers import Dense
from modules.stats import * 
from modules.PCA import * 
from modules.LDA import * 
from modules.read_data import * 
from modules.ROC import *

In [ ]:
path = '../../Assets/Isolated Digits'
data_class = ['2', '3', '4', '5', 'z']

C = []; D = []
B = []
for i in data_class:
	C.append(read_digit(path, i, 'train')[1])
	D.append(read_digit(path, i, 'dev')[1])
	B.append(read_digit(path, i, 'dev')[0])

In [ ]:
C_y = []
D_y = []

for i in range(5):
	C_y.append(i * np.ones(C[i].shape[0]))
	D_y.append(i * np.ones(D[i].shape[0]))

In [ ]:
X = np.concatenate(C, axis=0)
X_d = np.concatenate(D, axis=0)
Y = np.concatenate(C_y, axis=0)
Y_d = np.concatenate(D_y, axis=0)

In [ ]:
pca = False
lda = True
lda = ~pca & lda #if pca is true then lda is false

In [ ]:
if pca:
	pc = 15
	μ, Σ = stats(X)
	eigval, Q = PCA(Σ, pc)
	X = X @ Q
	X_d = X_d @ Q
if lda:
	pc = 15
	μ, Σ = stats(X)
	μk = []
	for c in range(5):
		μk.append(stats(X[Y == c])[0])
	μk = np.array(μk)
	v = LDA(X,Y,μ,μk,pc)
	X = np.abs(X @ v)
	X_d = np.abs(X_d @ v)

In [ ]:
classifier = Sequential()
classifier.add(Dense(512, input_shape=(X.shape[1],), kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(512, input_shape=(X.shape[1],), kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(5, kernel_initializer = 'uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.fit(X, label_binarize(Y, classes=[0,1,2,3,4]), batch_size = 50, epochs = 100)

In [ ]:
score = classifier.predict(X_d)
y_pred = np.argmax(score,axis=1)

In [ ]:
Yn_d = []
for c in range(5):
	Yn_d.append(c * np.ones(len(B[c])))
Yn_d = np.array(Yn_d)

yn_pred = np.zeros_like(Yn_d)
n = 0
for c in range(5):
	for i in range(len(B[c])):
		data = y_pred[n:n+B[c][i].shape[0]]
		n += B[c][i].shape[0]
		yn_pred[c][i] = statistics.mode(data)

In [ ]:
cm = confusion_matrix(yn_pred.flatten(), Yn_d.flatten())
accuracy = accuracy_score(Yn_d.flatten(),yn_pred.flatten())
ax = sns.heatmap(cm, annot = True, cmap ='plasma',linecolor ='black', linewidths = 1, fmt = '.0f', xticklabels = data_class, yticklabels = data_class)
ax.set_title(f'Accuracy = {accuracy}')

In [ ]:
TPR = 0; FPR = 0
for c in range(5):
	tpr, fpr = ROC(label_binarize(Y_d, classes=[0,1,2,3,4])[:,c], score[:,c])
	TPR += tpr/5
	FPR += fpr/5
plt.plot(FPR, TPR)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title("ROC")